In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pystan
import pandas as pd
import networkx as nx

plt.style.use('ggplot')
%matplotlib inline

In [2]:
# load nodes and edges from file
nodes = pd.read_csv('./data/got-s6-nodes.csv')
edges = pd.read_csv('./data/got-s6-edges.csv')

# initialize networkx graph object and add nodes and edges
G = nx.Graph(name='Game Of Thrones S06')
G.add_nodes_from(nodes['Id'])
edge_dict = [(x['Target'],x['Source'], {'weight' : x['Weight']}) for (_,x) in edges.iterrows()]
G.add_edges_from(edge_dict)

In [11]:
model_code="""
data {
        int<lower=1> N; // number of nodes
        matrix[N,N] X; // adjacency matrix
        int<lower=2> K; // number of groups
        vector[K] alpha; // prior for dirichlet distribution
} parameters {
        int z[N]; // group belonging, encoded as integers
        simplex[K] pi; // group sizes
        matrix<lower=0>[K,K] lambda; // rates between groups

} model {
        pi ~ dirichlet(alpha);
        for (k in 1:K) lambda[K, :] ~ gamma(1,0.5);
        z ~ categorical(pi);
        for (i in 1:N) {
            for (j in 1:N) {
                X[i,j] ~ poisson(lambda[z[i],z[j]]); // hmm... self links, ie. diagonal
            }
        }

}
"""

In [12]:
sm = pystan.StanModel(model_code=model_code)

ValueError: Failed to parse Stan model 'anon_model_8e61d6eb75446ab586adcdeed83dc971'. Error message:
SYNTAX ERROR, MESSAGE(S) FROM PARSER:

parameters or transformed parameters cannot be integer or integer array;  found declared type int, parameter name=z
Problem with declaration.
  error in 'unknown file name' at line 8, column 17
  -------------------------------------------------
     6:         vector[K] alpha; // prior for dirichlet distribution
     7: } parameters {
     8:         int z[N]; // group belonging, encoded as integers
                        ^
     9:         simplex[K] pi; // group sizes
  -------------------------------------------------

